In [1]:
from aicsimageio import AICSImage

In [2]:
img = AICSImage("/Users/danielt/Downloads/AICS-12_143.ome.tif")

In [3]:
data = img.get_image_data()
print(data.shape)
print(img.channel_names)
print(img.physical_pixel_sizes)

(1, 9, 60, 624, 924)
['CMDRP', 'EGFP', 'H3342', 'Bright_100X', 'SEG_STRUCT', 'SEG_Memb', 'SEG_DNA', 'CON_Memb', 'CON_DNA']
PhysicalPixelSizes(Z=0.29, Y=0.10833333333333332, X=0.10833333333333332)


In [9]:
import os
from aicsimageio.writers import OmeZarrWriter
# construct some per-channel lists to feed in to the writer.
# hardcoding to 9 for now
channel_colors = [0xff0000, 0x00ff00, 0x0000ff, 0xffff00, 0xff00ff, 0x00ffff, 0x880000, 0x008800, 0x000088]

mypath = "/Users/danielt/Documents/AICS-12_143.zarr"
# TODO: add generated thumbnail image with name "thumbnail" as 2d rgb data? or 3 channels?
# TODO compression?
OmeZarrWriter.save(
    data, 
    mypath,
    image_name="AICS-12_143",
    channel_names=img.channel_names,
    channel_colors=channel_colors,
    physical_pixel_sizes=img.physical_pixel_sizes,
)
print(os.listdir(mypath))
print(os.listdir(mypath + "/AICS-12_143"))


['AICS-12_143', '.zgroup']
['.zattrs', '.zgroup', '0', '1', '2']


In [12]:
from ome_zarr.reader import Multiscales, Reader
from ome_zarr.io import parse_url
reader = Reader(parse_url(mypath + "/AICS-12_143"))
node = list(reader())[0]
print(node.data[0].shape)
# assert node.data[0].chunks == ((1,), (2,), (1,), (128, 128), (128, 128))

(1, 9, 60, 624, 924)


In [13]:
# levels
print(len(node.data))

3


In [15]:
print(node.data[1].shape)
print(node.data[2].shape)
print(node.data[2][0].shape)


(1, 9, 60, 312, 462)
(1, 9, 60, 156, 231)
(9, 60, 156, 231)


In [16]:
import nbvv
level = 0
levelxyscale = 2**(level)
nbvv.volshow(node.data[level][0][0:3].compute(), spacing=(img.physical_pixel_sizes.X*levelxyscale,
        img.physical_pixel_sizes.Y*levelxyscale,
        img.physical_pixel_sizes.Z))

VolumeWidget(dimensions={'tile_width': 341, 'tile_height': 204, 'rows': 10, 'cols': 6, 'atlas_width': 2046, 'a…